# Requests e BeautifulSoup
Foi utilizado requests.get(url) para obter o conteúdo HTML da página, BeautifulSoup para analisar o conteúdo HTML obtido e soup.prettify() para imprimir a estrutura HTML de forma formatada, facilitando a leitura e compreensão dos elementos e suas propriedades. 

No entanto, mais abaixo, é possível perceber que o código principal utiliza Selenium para navegar e extrair dados de páginas que carregam conteúdo dinamicamente.



In [1]:
import requests
from bs4 import BeautifulSoup

# URL da página a ser analisada
url = 'https://www.coursera.org/courses?query=free&page=1'

# Fazer uma solicitação HTTP para a página
response = requests.get(url)

# Verificar se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Analisar o conteúdo HTML com BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Imprimir a estrutura HTML formatada
    print(soup.prettify())
else:
    print(f'Falha ao acessar a página. Status code: {response.status_code}')

<!DOCTYPE html>
<html dir="ltr" itemtype="http://schema.org" lang="en" xmlns:fb="http://ogp.me/ns/fb#">
 <head>
  <link crossorigin="" href="https://d3njjcbhbojbot.cloudfront.net" rel="preconnect"/>
  <meta content="IE=Edge,chrome=IE7" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta content="Coursera" property="og:site_name"/>
  <meta content="727836538,4807654" property="fb:admins"/>
  <meta content="823425307723964" property="fb:app_id"/>
  <meta content="Coursera" name="twitter:site"/>
  <meta content="Coursera" name="twitter:app:name:iphone"/>
  <meta content="Coursera" name="twitter:app:name:ipad"/>
  <meta content="Coursera" name="twitter:app:name:googleplay"/>
  <meta content="id736535961" name="twitter:app:id:iphone"/>
  <meta content="id736535961" name="twitter:app:id:ipad"/>
  <meta content="org.coursera.android" name="twitter:app:id:googleplay"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://d3njjcbhbojbot.clou

# Código Selenium
Foi utilizado Selenium para automatizar a navegação em um navegador web, especificamente no site do Coursera, com o objetivo de coletar informações sobre cursos gratuitos. 

As informações coletadas foram armazenadas na pasta scrapped_datasets, em um arquivo chamado "courses.csv", contendo as colunas:

'Course Name' (Nome do curso), 

'Title' (Nome do curso), 

'Image URL'  (URL da imagem do curso), 

'Course Link' (URL para a página do curso), 

'Institution' (Instituição que disponibiliza o curso dentro da plataforma da Coursera), 

'Skills' (Habilidades), 

'Rating' (Nota do curso), 

'Number of Reviews' (Quantidade de avaliações do curso feitas pelos usuários), 

'Level' (Nível de dificuldade do curso), 

'Course Type' (Se é um projeto guiado, curso, especialização ou certificação) e

 'Duration' (Duração do curso).

In [ ]:
import os
import requests
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Configurações do Chrome
options = Options()
# options.add_argument('--headless')  # Descomente para rodar sem abrir o navegador

# Inicialização do WebDriver
driver = webdriver.Chrome(options=options)

# Diretório raiz para salvar os cursos
root_dir = 'scrapped_datasets'
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

# Função para extrair os dados da página
def extract_courses():
    course_cards = driver.find_elements(By.CSS_SELECTOR, 'li.cds-9')
    
    if not course_cards:
        return False  # Indica que a página não contém cursos

    for card in course_cards:
        try:
            title_element = card.find_element(By.CSS_SELECTOR, 'h3.cds-CommonCard-title')
            title = title_element.text.strip() if title_element else 'Título não encontrado'

            image_url = 'Imagem não encontrada'
            try:
                image_element = WebDriverWait(card, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'img[src*="https://"]'))
                )
                image_url = image_element.get_attribute('src') if image_element else 'Imagem não encontrada'
            except Exception:
                pass

            link_element = card.find_element(By.CSS_SELECTOR, 'a.cds-CommonCard-titleLink')
            course_link = link_element.get_attribute('href') if link_element else 'Link não encontrado'

            institution = 'Instituição não encontrada'
            try:
                institution_element = card.find_element(By.CSS_SELECTOR, 'p.cds-ProductCard-partnerNames')
                institution = institution_element.text.strip() if institution_element else 'Instituição não encontrada'
            except Exception:
                pass

            skills = 'Habilidades não encontradas'
            try:
                skills_element = card.find_element(By.CSS_SELECTOR, 'div.cds-CommonCard-bodyContent')
                skills = skills_element.text.strip() if skills_element else 'Habilidades não encontradas'
            except Exception:
                pass

            rating = "Avaliação não encontrada"
            num_reviews = "Número de avaliações não encontrado"
            level = "Nível não encontrado"
            course_type = "Tipo do curso não encontrado"
            duration = "Duração não encontrada"

            try:
                rating_element = card.find_element(By.CSS_SELECTOR, 'div.cds-ProductCard-footer')
                rating_text = rating_element.text.strip()

                rating_match = re.search(r"(\d,\d)\s+classificação", rating_text)
                if rating_match:
                    rating = rating_match.group(1)

                reviews_match = re.search(r"([\d,.]+\s*\w*)\s+avaliações", rating_text)
                if reviews_match:
                    num_reviews = reviews_match.group(1)

                level_match = re.search(r"(Misto|Básico|Intermediário|Avançado)", rating_text)
                if level_match:
                    level = level_match.group(1)

                course_type_match = re.search(r"(projeto guiado|Curso|Especialização|Certificação)", rating_text)
                if course_type_match:
                    course_type = course_type_match.group(1)

                level_duration_parts = rating_text.split('·')
                if len(level_duration_parts) >= 4:
                    duration = level_duration_parts[3].strip()

            except Exception:
                pass

            # Nome do diretório do curso
            course_name = title.replace(' ', '_').replace('/', '_').replace('\\', '_')

            # Salvar os dados do curso
            save_course_data_csv(course_name, title, image_url, course_link, institution, skills, rating, num_reviews, level, course_type, duration, root_dir)

        except Exception as e:
            print(f'Erro ao processar o card: {e}')
    
    return True  # Indica que a página continha cursos

# Função para salvar os dados do curso em um arquivo .csv
def save_course_data_csv(course_name, title, image_url, course_link, institution, skills, rating, num_reviews, level, course_type, duration, root_dir):
    csv_file_path = os.path.join(root_dir, 'courses.csv')
    headers = ['Course Name', 'Title', 'Image URL', 'Course Link', 'Institution', 'Skills', 'Rating', 'Number of Reviews', 'Level', 'Course Type', 'Duration']
    
    file_exists = os.path.isfile(csv_file_path)
    
    try:
        with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file, delimiter=';')
            if not file_exists:
                writer.writerow(headers)
            
            writer.writerow([course_name, title, image_url, course_link, institution, skills.replace("Habilidades que você terá: ", ""), rating, num_reviews, level, course_type, duration])
        print(f'Dados de "{course_name}" salvos com sucesso no arquivo CSV!')
    except Exception as e:
        print(f'Erro ao salvar os dados para "{course_name}" no arquivo CSV: {e}')

# Loop para navegar pelas páginas via URL
page = 1
while True:
    url = f'https://www.coursera.org/courses?query=free&page={page}'
    print(f"Acessando: {url}")
    
    try:
        driver.get(url)
        
        # Verifica se a página contém a mensagem de "Nenhum resultado encontrado"
        if "Não encontramos nenhuma correspondência exata relacionada " in driver.page_source:
            print(f"Página {page} sem cursos. Encerrando o scraping.")
            break

        # Espera os cursos carregarem
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.cds-9')))

        # Extração dos cursos
        if not extract_courses():
            print(f"Página {page} sem cursos. Encerrando o scraping.")
            break

    except Exception as e:
        print(f"Erro ao acessar a página {page}: {e}")
        break

    page += 1
    time.sleep(3)  # Pausa para evitar sobrecarga no servidor

# Fechar o navegador
driver.quit()


Acessando: https://www.coursera.org/courses?query=free&page=1
Dados de "Build_a_free_website_with_WordPress" salvos com sucesso no arquivo CSV!
Dados de "Business_Analysis_&_Process_Management" salvos com sucesso no arquivo CSV!
Dados de "Getting_Started_with_Microsoft_Excel" salvos com sucesso no arquivo CSV!
Dados de "Inglês_para_interações_comuns_no_local_de_trabalho:_Nível_básico" salvos com sucesso no arquivo CSV!
Dados de "Python_para_ciência_de_dados,_IA_e_desenvolvimento" salvos com sucesso no arquivo CSV!
Dados de "Primeiro_passo_coreano" salvos com sucesso no arquivo CSV!
Dados de "Inglês_para_desenvolvimento_de_carreira" salvos com sucesso no arquivo CSV!
Dados de "Pensamento_criativo:_técnicas_e_ferramentas_para_o_sucesso" salvos com sucesso no arquivo CSV!
Dados de "Mercados_Financeiros" salvos com sucesso no arquivo CSV!
Dados de "Introdução_à_Fisiologia_Humana" salvos com sucesso no arquivo CSV!
Dados de "Investment_Risk_Management" salvos com sucesso no arquivo CSV!
Dad